[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/notebookOBDTC.ipynb)



# Object-detection made easy
We will learn how to easily train an object detection model from a list of pre-trained models with the dataset you created on the picsell-IA platform.

In [ ]:
!git clone https://github.com/BouleJaune/picselliaT
%cd picselliaT/
!pip install picsellia
%tensorflow_version 1.x

## Imports

In [1]:
import sys
import os
sys.path.append("slim")
from picsellia import Client
import main
from util.infer import infer
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.







## List of possible models : 

- Mask RCNN, a segmentation model. String variable: 'mask_rcnn'
- Faster RCNN an accurate but slow object detection model. String variable: 'faster_rcnn'
- SSD Inception, a fast but less accurate object detection model. String variable : 'ssd_inception'

## Setup

We need to start setting up some variables before anything else.

In [2]:
project_token = "d8e83b34-352e-41b1-9329-03a791f8d4ef" # Token from the picsell-IA platform
api_token = "5890ad3e6a9701cd123ef91b69ad396de8c2d20a"
model_name = "base_maskrcnn" # Name your to-be trained model
annotation_type = "polygon" # Chose the type of annotation used

batch_size = 1
learning_rate = None #You can let this value to None
nb_steps = 10

## Client initialisation and data pre-processing

We communicate with the platform to create a new model and get the images and annotations.
With this we can generate the label map, smartly split our data then create the TFRecord files which will be used as input for the model. 

In [3]:
clt = Client(api_token=api_token, host="https://backstage.picsellia.com/sdk/")
clt.init_project(project_token=project_token)
clt.init_model(model_name)

clt.dl_annotations()
clt.generate_labelmap()
clt.local_pic_save()

Welcome Xenio3, Please initialize your project with init_project() and the project token provided on the platform
Available project : 

	 mask-classification
	 careden_train
{'project_token': 'd8e83b34-352e-41b1-9329-03a791f8d4ef'}
Welcome to Picsell.ia Client, this project_token is linked to your project : careden_train
This is a detection project
Here is the current state of your project:

---------------
4 training version(s) for Network named : car_bruises_mask_rcnn
---------------
	 For training id 0:

		 Train Test Set repartition : DONE
		 Visual results uploaded to Picsell.ia : DONE
		 Training logs uploaded to Picsell.ia : DONE
		 Model usable from Picsell.ia : DONE
	 For training id 1:

		 Train Test Set repartition : DONE
		 Visual results uploaded to Picsell.ia : NOT DONE
		 Training logs uploaded to Picsell.ia : DONE
		 Model usable from Picsell.ia : DONE
	 For training id 2:

		 Train Test Set repartition : NOT DONE
		 Visual results uploaded to Picsell.ia : NOT DONE
		 T

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=====                                             ]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========                                         ]

...

[=============================================     ]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================================================= ]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Annotations pulled ...
Generating labelmap ...
Label_map.pbtxt crée @ d8e83b34-352e-41b1-9329-03a791f8d4ef/4b0d7fdd-6e9b-443a-a44e-a8b1508ce4bc/3/label_map.pbtxt
416 Images used for training, 104 Images used for validation
520 files were already on your machine
 0 PNG images have been downloaded to your machine
Sending repartition to Picsell.ia backend
Repartition send ..


In [ ]:
main.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir,
                         tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)

Here we edit the base protobuf configuration of our model with our parameters. 
We check if it the first training on this model to see if we should train from a previous checkpoint or not.

In [ ]:
main.edit_config(model_selected=clt.model_selected, config_output_dir=clt.config_dir,
            record_dir=clt.record_dir, 
            label_map_path=clt.label_path, 
            num_steps=nb_steps,
            batch_size=batch_size, 
            learning_rate=learning_rate,
            annotation_type=annotation_type,
            size=500,
            eval_number = len(clt.eval_list))

## Training

We can launch the training... and it's as easy as just telling the fonction where is the configuration file and where we want the checkpoints and records to be saved !

In [ ]:
main.train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)

Now that the training has ended we want to send the logs to our dashboard so we can neatly see the sweet decrease of the loss. 

In [ ]:
dict_log = main.tfevents_to_dict(path=clt.checkpoint_dir)
clt.send_logs(dict_log)

## Exporting and infering

The model is trained but we still need to export it to a Tensorflow graph proto to use it.

In [ ]:
main.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model, 
                       pipeline_config_path=clt.config_dir)

Now we will use the exported model to do some inference on our evaluation set, then send the results to the dashboard.
You can set the minimum confidence treshold at which we keep the bounding boxes to the value you like the most.

In [ ]:
min_score_thresh = 0.3 
infer(clt.eval_list, exported_model_dir=clt.exported_model, 
          label_map_path=clt.label_path, results_dir=clt.results_dir, min_score_thresh=min_score_thresh, num_infer=16)
clt.send_examples()

In [ ]:
clt.send_weights(clt.exported_model)

In [ ]:
main.evaluate(clt.log_dir, clt.config_dir, clt.checkpoint_dir)